# Realistica DDPM - Google Colab Setup

This notebook sets up the Realistica diffusion model package on Google Colab.

**Steps:**
1. Clone the repository
2. Install dependencies
3. Run one of the training notebooks

**GPU Runtime Required**: Go to Runtime → Change runtime type → Select GPU (T4 is fine)

## Step 1: Clone Repository

In [ ]:
# Clone the repository
!git clone https://github.com/luisdiaz1997/Difussion.git
%cd Difussion

## Step 2: Install Package

In [ ]:
# Install the package in development mode
!pip install -e .

# Verify installation
import torch
import realistica

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Realistica version: {realistica.__version__}")

## Step 3: Choose Your Experiment

Now you can run any of the notebooks:

### Option A: Standard DDPM on MNIST (Fast, ~30 min)
```python
%run notebooks/mnist_diffusion.ipynb
```

### Option B: DDPM with GP Noise on MNIST (Fast, ~30 min)
```python
%run notebooks/mnist_diffusion_matern32.ipynb
```

### Option C: Standard DDPM on CelebA (Slow, ~4 hours)
```python
%run notebooks/celeba_diffusion.ipynb
```

### Option D: DDPM with GP Noise on CelebA (Slow, ~4 hours)
```python
%run notebooks/celeba_diffusion_matern32.ipynb
```

**Recommendation**: Start with MNIST to verify everything works!

## Quick Test: MNIST with GP Noise (Reduced Epochs)

In [ ]:
# Quick test with just 10 epochs
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

from realistica import (
    NoiseScheduler,
    UNet,
    ImageGPNoiseSampler,
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Quick config
config = {
    'image_size': 28,
    'batch_size': 128,
    'num_epochs': 10,  # Just 10 epochs for quick test
    'learning_rate': 2e-4,
    'num_timesteps': 1000,
    'gp_lengthscale': 0.15,
    'gp_num_features': 512,
}

# Load data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=2)

print(f"Dataset loaded: {len(train_dataset)} images")

In [ ]:
# Initialize model
noise_scheduler = NoiseScheduler(
    num_timesteps=config['num_timesteps'],
    beta_start=0.0001,
    beta_end=0.02,
    schedule_type='linear',
    device=device
)

model = UNet(
    in_channels=1,
    out_channels=1,
    base_channels=64,
    channel_multipliers=(1, 2, 2, 2),
    num_res_blocks=2,
    time_emb_dim=256,
    attention_levels=(False, False, True, True),
    dropout=0.1
).to(device)

gp_noise_sampler = ImageGPNoiseSampler(
    height=config['image_size'],
    width=config['image_size'],
    lengthscale=config['gp_lengthscale'],
    variance=1.0,
    num_features=config['gp_num_features'],
    kernel_type='matern32',
    device=device
)

optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'])
criterion = nn.MSELoss()

print(f"Model initialized with {sum(p.numel() for p in model.parameters()):,} parameters")

In [ ]:
# Train
model.train()
losses = []

for epoch in range(config['num_epochs']):
    epoch_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config['num_epochs']}")
    
    for images, _ in progress_bar:
        images = images.to(device)
        batch_size = images.shape[0]
        
        t = noise_scheduler.sample_timesteps(batch_size)
        noisy_images, noise = noise_scheduler.add_gp_noise(images, t, gp_noise_sampler=gp_noise_sampler)
        
        predicted_noise = model(noisy_images, t)
        loss = criterion(predicted_noise, noise)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        losses.append(loss.item())
        progress_bar.set_postfix({'loss': f'{loss.item():.6f}'})
    
    print(f"Epoch {epoch+1} - Avg Loss: {epoch_loss/len(train_loader):.6f}")

print("\nTraining complete!")

In [ ]:
# Generate samples
@torch.no_grad()
def sample(num_samples=64):
    model.eval()
    x = gp_noise_sampler.sample(num_samples, 1)
    
    for t in tqdm(reversed(range(noise_scheduler.num_timesteps)), desc='Sampling'):
        t_batch = torch.tensor([t] * num_samples, device=device)
        predicted_noise = model(x, t_batch)
        
        alpha = noise_scheduler.alphas[t]
        alpha_cumprod = noise_scheduler.alphas_cumprod[t]
        beta = noise_scheduler.betas[t]
        
        noise = gp_noise_sampler.sample(num_samples, 1) if t > 0 else torch.zeros_like(x)
        
        x = (1 / torch.sqrt(alpha) * (x - (beta / torch.sqrt(1 - alpha_cumprod)) * predicted_noise) + 
             torch.sqrt(beta) * noise)
    
    model.train()
    return x

samples = sample(64)

# Visualize
grid = make_grid(samples, nrow=8, normalize=True, value_range=(-1, 1))
plt.figure(figsize=(12, 12))
plt.imshow(grid.permute(1, 2, 0).cpu().numpy(), cmap='gray')
plt.axis('off')
plt.title('Generated MNIST Digits (GP Noise - 10 epochs)')
plt.show()

## Success!

If you see generated digits above, everything is working! 

For better results, train for more epochs (50+ recommended).

**Next Steps:**
- Navigate to the `notebooks/` folder and open any notebook
- Modify hyperparameters and experiment
- Try CelebA for face generation (requires more GPU time)